# Read Data Sample

In [1]:
import pandas as pd
import numpy as np
import os
from collections import namedtuple
pd.set_option("display.max_rows",35)
%matplotlib inline

In [2]:
class dataset:
    kdd_train_2labels = pd.read_pickle("dataset/kdd_train_2labels.pkl")
    kdd_test_2labels = pd.read_pickle("dataset/kdd_test_2labels.pkl")
    
    kdd_train_5labels = pd.read_pickle("dataset/kdd_train_5labels.pkl")
    kdd_test_5labels = pd.read_pickle("dataset/kdd_test_5labels.pkl")
    

In [3]:
dataset.kdd_train_2labels.shape

(125973, 124)

In [4]:
dataset.kdd_test_2labels.shape

(22544, 124)

In [5]:
from sklearn import model_selection as ms
from sklearn import preprocessing as pp

class preprocess:
    
    output_columns_2labels = ['is_Attack','is_Normal']
    
    x_input = dataset.kdd_train_2labels.drop(output_columns_2labels, axis = 1)
    y_output = dataset.kdd_train_2labels.loc[:,output_columns_2labels]

    x_test_input = dataset.kdd_test_2labels.drop(output_columns_2labels, axis = 1)
    y_test = dataset.kdd_test_2labels.loc[:,output_columns_2labels]

    ss = pp.StandardScaler()

    x_train = ss.fit_transform(x_input)
    x_test = ss.transform(x_test_input)

    y_train = y_output.values
    y_test = y_test.values

    x_train = np.hstack((x_train, y_train))
    x_test = np.hstack((x_test, np.random.normal(size = (x_test.shape[0], y_train.shape[1]))))
    #x_test = np.hstack((x_test, y_test))
    
preprocess.x_train.std()

0.99186991653217393

In [6]:
import tensorflow as tf


In [7]:
class network(object):
    
    input_dim = 124
    classes = 2
    hidden_encoder_dim = 124
    hidden_layers = 1
    latent_dim = 10

    hidden_decoder_dim = 124
    lam = 0.01
    
    def __init__(self, classes, hidden_layers, num_of_features):
        self.classes = classes
        self.hidden_layers = hidden_layers
        self.latent_dim = num_of_features
            
    def build_layers(self):
        tf.reset_default_graph()
        #learning_rate = tf.Variable(initial_value=0.001)

        input_dim = self.input_dim
        classes = self.classes
        hidden_encoder_dim = self.hidden_encoder_dim
        hidden_layers = self.hidden_layers
        latent_dim = self.latent_dim
        hidden_decoder_dim = self.hidden_decoder_dim
        lam = self.lam
        
        with tf.variable_scope("Input"):
            self.x = tf.placeholder("float", shape=[None, input_dim])
            self.y_ = tf.placeholder("float", shape=[None, classes])
            self.keep_prob = tf.placeholder("float")
        
        with tf.variable_scope("Layer_Encoder"):

            hidden_encoder = tf.layers.dense(self.x, hidden_encoder_dim, activation = tf.nn.relu, kernel_regularizer=tf.nn.l2_loss)
            hidden_encoder = tf.nn.dropout(hidden_encoder, self.keep_prob)
            for h in range(hidden_layers - 1):
                hidden_encoder = tf.layers.dense(hidden_encoder, hidden_encoder_dim, activation = tf.nn.relu, kernel_regularizer=tf.nn.l2_loss)
                hidden_encoder = tf.nn.dropout(hidden_encoder, self.keep_prob)
                
        with tf.variable_scope("Layer_Mean"):
            mu_encoder = tf.layers.dense(hidden_encoder, latent_dim, activation = None, kernel_regularizer=tf.nn.l2_loss)

        with tf.variable_scope("Layer_Variance"):
            logvar_encoder = tf.layers.dense(hidden_encoder, latent_dim, activation = None, kernel_regularizer=tf.nn.l2_loss)

        with tf.variable_scope("Sampling_Distribution"):
            # Sample epsilon
            epsilon = tf.random_normal(tf.shape(logvar_encoder), mean=0, stddev=1, name='epsilon')

            # Sample latent variable
            std_encoder = tf.exp(0.5 * logvar_encoder)
            z = mu_encoder + tf.multiply(std_encoder, epsilon)
            
            #tf.summary.histogram("Sample_Distribution", z)

        with tf.variable_scope("Layer_Decoder"):
            hidden_decoder = tf.layers.dense(z, hidden_decoder_dim, activation = tf.nn.relu, kernel_regularizer=tf.nn.l2_loss)
            hidden_decoder = tf.nn.dropout(hidden_decoder, self.keep_prob)
            for h in range(hidden_layers - 1):
                hidden_decoder = tf.layers.dense(hidden_decoder, hidden_decoder_dim, activation = tf.nn.relu, kernel_regularizer=tf.nn.l2_loss)
                hidden_decoder = tf.nn.dropout(hidden_decoder, self.keep_prob)
                
        with tf.variable_scope("Layer_Reconstruction"):
            self.x_hat = tf.layers.dense(hidden_decoder, input_dim, activation = None)
            
            self.y = tf.slice(self.x_hat, [0,input_dim-2], [-1,-1])

        with tf.variable_scope("Loss"):
            
            self.regularized_loss = tf.losses.mean_squared_error(self.x, self.x_hat) #tf.reduce_mean((BCE + KLD + softmax_loss) * lam)
            loss = tf.where(tf.is_nan(self.regularized_loss), 1e-2, self.regularized_loss)
            
            correct_prediction = tf.equal(tf.argmax(self.y, 1), tf.argmax(self.y_, 1))
            self.tf_accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name = "Accuracy")

        with tf.variable_scope("Optimizer"):
            learning_rate=1e-2
            optimizer = tf.train.AdamOptimizer(learning_rate)
            gradients, variables = zip(*optimizer.compute_gradients(self.regularized_loss))
            gradients = [
                None if gradient is None else tf.clip_by_value(gradient, -1, 1)
                for gradient in gradients]
            self.train_op = optimizer.apply_gradients(zip(gradients, variables))
            #self.train_op = optimizer.minimize(self.regularized_loss)
            
        # add op for merging summary
        #self.summary_op = tf.summary.merge_all()
        self.pred = tf.argmax(self.y, axis = 1)
        self.actual = tf.argmax(self.y_, axis = 1)

        # add Saver ops
        self.saver = tf.train.Saver()
        

In [10]:
import collections

class Train:    
    
    result = namedtuple("score", ['epoch', 'no_of_features','hidden_layers','train_score', 'test_score'])

    predictions = {}

    results = []
    best_acc = 0
    
    def train(epochs, net, h,f):
        batch_iterations = 200
        os.makedirs("dataset/tf_vae_only_nsl_kdd/hidden layers_{}_features count_{}".format(epochs,h,f),
                    exist_ok = True)
        with tf.Session() as sess:
            #summary_writer_train = tf.summary.FileWriter('./logs/kdd/VAE/training', graph=sess.graph)
            #summary_writer_valid = tf.summary.FileWriter('./logs/kdd/VAE/validation')

            sess.run(tf.global_variables_initializer())
            
            Train.best_acc = 0
            for epoch in range(1, (epochs+1)):
                #print("Step {} | Training Loss:".format(epoch), end = " " )
                x_train, x_valid, y_train, y_valid, = ms.train_test_split(preprocess.x_train, 
                                                                          preprocess.y_train, 
                                                                          test_size=0.1)
                batch_indices = np.array_split(np.arange(x_train.shape[0]), 
                                           batch_iterations)
                                                                          
                for i in batch_indices:
                    _, train_loss = sess.run([net.train_op, 
                                                           net.regularized_loss, 
                                                           ], #net.summary_op
                                                          feed_dict={net.x: x_train[i,:], 
                                                                     net.y_: y_train[i,:], 
                                                                     net.keep_prob:1})
                    
                    if((train_loss > 1e4 or np.isnan(train_loss) ) and epoch > 1):
                        print("Step {} | High Training Loss: {:.6f} ... Restoring Net".format(epoch, train_loss))
                        net.saver.restore(sess, 
                                          tf.train.latest_checkpoint('dataset/tf_vae_only_nsl_kdd/hidden layers_{}_features count_{}'
                                                                     .format(epochs,h,f)))
                    
                    #summary_writer_train.add_summary(summary_str, epoch)
                    #if(train_loss > 1e9):
                    
                    #print("{:.6f}".format(train_loss), end = ", " )
                    
                #print("")
                valid_loss, valid_accuracy = sess.run([net.regularized_loss, net.tf_accuracy], feed_dict={net.x: x_valid, 
                                                                     net.y_: y_valid, 
                                                                     net.keep_prob:1})
                    
                
                accuracy, test_loss, pred_value, actual_value, y_pred = sess.run([net.tf_accuracy, net.regularized_loss, 
                                                               net.pred, 
                                                               net.actual, net.y], 
                                                              feed_dict={net.x: preprocess.x_test, 
                                                                         net.y_: preprocess.y_test, 
                                                                         net.keep_prob:1})
                #print("*************** \n")
                print("Step {} | Training Loss: {:.6f} | Test Loss: {:6f} | Test Accuracy: {:.6f}".format(epoch, train_loss, test_loss, accuracy))
                #print("*************** \n")
                #print("Accuracy on Test data: {}".format(accuracy))

                
                if accuracy > Train.best_acc:
                    Train.best_acc = accuracy
                    Train.pred_value = pred_value
                    Train.actual_value = actual_value
                    Train.best_parameters = "Hidden Layers:{}, Features Count:{}".format(h, f)
                    
                    #net.saver.save(sess, "dataset/tf_vae_only_nsl_kdd_hidden layers_{}_features count_{}".format(epochs,h,f))
                    #Train.results.append(Train.result(epochs, f, h,valid_accuracy, accuracy))
                    #curr_pred = pd.DataFrame({"Attack_prob":y_pred[:,-2], "Normal_prob":y_pred[:, -1]})
                    #Train.predictions.update({"{}_{}_{}".format(epochs,f,h):curr_pred})
                    
                    net.saver.save(sess, 
                                   "dataset/tf_vae_only_nsl_kdd/hidden layers_{}_features count_{}/model"
                                   .format(epochs,h,f), 
                                   global_step = epoch, 
                                   write_meta_graph=False)
                    
                    curr_pred = pd.DataFrame({"Attack_prob":y_pred[:,-2], "Normal_prob":y_pred[:, -1]})
                    Train.predictions.update({"{}_{}_{}".format(epochs,f,h):
                                              (curr_pred, 
                                               Train.result(epochs, f, h,valid_accuracy, accuracy))})

            

In [ ]:
import itertools
class Hyperparameters:
#    features_arr = [2, 4, 8, 16, 32, 64, 128, 256]
#    hidden_layers_arr = [2, 4, 6, 10]
    features_arr = [4, 8, 16, 32]
    hidden_layers_arr = [2, 4]

    epochs = [100]
    
    for e, h, f in itertools.product(epochs, hidden_layers_arr, features_arr):
        print("Current Layer Attributes - epochs:{} hidden layers:{} features count:{}".format(e,h,f))
        n = network(2,h,f)
        n.build_layers()
        Train.train(e, n, h,f)
        

Current Layer Attributes - epochs:100 hidden layers:2 features count:4
Step 1 | Training Loss: 0.257782 | Test Loss: 1.306877 | Test Accuracy: 0.785309
Step 2 | Training Loss: 0.443546 | Test Loss: 1.143690 | Test Accuracy: 0.774086
Step 3 | Training Loss: 0.223919 | Test Loss: 1.112429 | Test Accuracy: 0.790898
Step 4 | Training Loss: 0.178301 | Test Loss: 1.079536 | Test Accuracy: 0.766013
Step 5 | Training Loss: 2.047094 | Test Loss: 1.234312 | Test Accuracy: 0.778123
Step 6 | Training Loss: 0.209088 | Test Loss: 1.087026 | Test Accuracy: 0.750532
Step 7 | Training Loss: 0.237695 | Test Loss: 1.134360 | Test Accuracy: 0.781716
Step 8 | Training Loss: 0.444343 | Test Loss: 1.408025 | Test Accuracy: 0.772179
Step 9 | Training Loss: 0.305551 | Test Loss: 1.361274 | Test Accuracy: 0.770848
Step 10 | Training Loss: 0.090387 | Test Loss: 1.582946 | Test Accuracy: 0.775861
Step 11 | Training Loss: 0.380696 | Test Loss: 1.535866 | Test Accuracy: 0.767166
Step 12 | Training Loss: 0.227589 | 

Step 99 | Training Loss: 0.330982 | Test Loss: 1.815522 | Test Accuracy: 0.746141
Step 100 | Training Loss: 0.346559 | Test Loss: 1.785671 | Test Accuracy: 0.725027
Current Layer Attributes - epochs:100 hidden layers:2 features count:8
Step 1 | Training Loss: 0.502608 | Test Loss: 1.447330 | Test Accuracy: 0.771380
Step 2 | Training Loss: 0.312036 | Test Loss: 1.450041 | Test Accuracy: 0.768275
Step 3 | Training Loss: 0.310575 | Test Loss: 1.290299 | Test Accuracy: 0.782559
Step 4 | Training Loss: 0.383066 | Test Loss: 1.274287 | Test Accuracy: 0.781139
Step 5 | Training Loss: 0.474309 | Test Loss: 1.284025 | Test Accuracy: 0.791962
Step 6 | Training Loss: 0.240601 | Test Loss: 1.322343 | Test Accuracy: 0.623625
Step 7 | Training Loss: 0.203866 | Test Loss: 1.485759 | Test Accuracy: 0.693577
Step 8 | Training Loss: 0.253571 | Test Loss: 1.684582 | Test Accuracy: 0.855350
Step 9 | Training Loss: 1.796509 | Test Loss: 1.625471 | Test Accuracy: 0.808951
Step 10 | Training Loss: 0.217177 |

Step 97 | Training Loss: 0.143988 | Test Loss: 1.649950 | Test Accuracy: 0.689452
Step 98 | Training Loss: 0.119415 | Test Loss: 1.719171 | Test Accuracy: 0.705066
Step 99 | High Training Loss: nan ... Restoring Net
INFO:tensorflow:Restoring parameters from dataset/tf_vae_only_nsl_kdd/hidden layers_100_features count_2/model-8
Step 99 | Training Loss: 0.242234 | Test Loss: 1.651126 | Test Accuracy: 0.856813
Step 100 | Training Loss: 0.484145 | Test Loss: 1.606069 | Test Accuracy: 0.827803
Current Layer Attributes - epochs:100 hidden layers:2 features count:16
Step 1 | Training Loss: 0.510013 | Test Loss: 1.598696 | Test Accuracy: 0.783623
Step 2 | Training Loss: 0.391795 | Test Loss: 1.577777 | Test Accuracy: 0.804737
Step 3 | Training Loss: 0.399385 | Test Loss: 1.496396 | Test Accuracy: 0.763707
Step 4 | High Training Loss: 11991.155273 ... Restoring Net
INFO:tensorflow:Restoring parameters from dataset/tf_vae_only_nsl_kdd/hidden layers_100_features count_2/model-2
Step 4 | Training 

Step 73 | Training Loss: 0.256795 | Test Loss: 1.494878 | Test Accuracy: 0.721168
Step 74 | Training Loss: 0.281175 | Test Loss: 1.500815 | Test Accuracy: 0.732656
Step 75 | Training Loss: 0.323677 | Test Loss: 1.406467 | Test Accuracy: 0.738556
Step 76 | Training Loss: 0.265714 | Test Loss: 1.395062 | Test Accuracy: 0.750621
Step 77 | High Training Loss: 27953899503616.000000 ... Restoring Net
INFO:tensorflow:Restoring parameters from dataset/tf_vae_only_nsl_kdd/hidden layers_100_features count_2/model-37
Step 77 | High Training Loss: 3030633.250000 ... Restoring Net
INFO:tensorflow:Restoring parameters from dataset/tf_vae_only_nsl_kdd/hidden layers_100_features count_2/model-37
Step 77 | Training Loss: 0.403821 | Test Loss: 1.490642 | Test Accuracy: 0.789656
Step 78 | High Training Loss: 2382637299344080896.000000 ... Restoring Net
INFO:tensorflow:Restoring parameters from dataset/tf_vae_only_nsl_kdd/hidden layers_100_features count_2/model-37
Step 78 | Training Loss: 0.394298 | Test

Step 59 | Training Loss: 0.903009 | Test Loss: 1.983041 | Test Accuracy: 0.430758
Step 60 | Training Loss: 1.022023 | Test Loss: 1.984755 | Test Accuracy: 0.430758
Step 61 | Training Loss: 1.086197 | Test Loss: 1.984707 | Test Accuracy: 0.430758
Step 62 | Training Loss: 0.842892 | Test Loss: 1.983836 | Test Accuracy: 0.430758
Step 63 | Training Loss: 0.756890 | Test Loss: 1.983503 | Test Accuracy: 0.430758
Step 64 | Training Loss: 0.839238 | Test Loss: 1.983868 | Test Accuracy: 0.430758
Step 65 | Training Loss: 2.336302 | Test Loss: 1.983900 | Test Accuracy: 0.430758
Step 66 | Training Loss: 0.811611 | Test Loss: 1.984010 | Test Accuracy: 0.430758
Step 67 | Training Loss: 0.804862 | Test Loss: 1.984234 | Test Accuracy: 0.430758
Step 68 | Training Loss: 0.931544 | Test Loss: 1.985030 | Test Accuracy: 0.430758
Step 69 | Training Loss: 0.964546 | Test Loss: 1.983262 | Test Accuracy: 0.430758
Step 70 | Training Loss: 0.820547 | Test Loss: 1.983639 | Test Accuracy: 0.430758
Step 71 | Traini

Step 57 | Training Loss: 0.675939 | Test Loss: 1.815386 | Test Accuracy: 0.701606
Step 58 | Training Loss: 0.673367 | Test Loss: 1.805695 | Test Accuracy: 0.737890
Step 59 | Training Loss: 0.816305 | Test Loss: 1.801827 | Test Accuracy: 0.721256
Step 60 | Training Loss: 0.704898 | Test Loss: 1.882756 | Test Accuracy: 0.762110
Step 61 | Training Loss: 0.741233 | Test Loss: 1.826596 | Test Accuracy: 0.757807
Step 62 | Training Loss: 0.595942 | Test Loss: 1.814183 | Test Accuracy: 0.768719
Step 63 | Training Loss: 0.505931 | Test Loss: 1.812322 | Test Accuracy: 0.754037
Step 64 | Training Loss: 0.659045 | Test Loss: 1.823953 | Test Accuracy: 0.753992
Step 65 | Training Loss: 1.712160 | Test Loss: 1.860344 | Test Accuracy: 0.740729
Step 66 | Training Loss: 0.738194 | Test Loss: 1.966749 | Test Accuracy: 0.474849
Step 67 | Training Loss: 0.689882 | Test Loss: 1.966250 | Test Accuracy: 0.474450
Step 68 | High Training Loss: 2419075888744763490304.000000 ... Restoring Net
INFO:tensorflow:Rest

Step 48 | Training Loss: 0.689503 | Test Loss: 1.900541 | Test Accuracy: 0.864399
Step 49 | Training Loss: 1.221022 | Test Loss: 1.900569 | Test Accuracy: 0.853220
Step 50 | Training Loss: 0.755010 | Test Loss: 1.900388 | Test Accuracy: 0.850293
Step 51 | Training Loss: 1.648639 | Test Loss: 1.935231 | Test Accuracy: 0.590978
Step 52 | Training Loss: 0.854854 | Test Loss: 1.934366 | Test Accuracy: 0.584413
Step 53 | Training Loss: 0.825475 | Test Loss: 1.935495 | Test Accuracy: 0.586586
Step 54 | Training Loss: 0.822263 | Test Loss: 1.933143 | Test Accuracy: 0.585610
Step 55 | Training Loss: 0.698313 | Test Loss: 1.937743 | Test Accuracy: 0.595014
Step 56 | Training Loss: 0.843985 | Test Loss: 1.935514 | Test Accuracy: 0.586586
Step 57 | Training Loss: 0.751591 | Test Loss: 1.934569 | Test Accuracy: 0.589824
Step 58 | Training Loss: 0.795035 | Test Loss: 1.934489 | Test Accuracy: 0.596877
Step 59 | Training Loss: 0.689297 | Test Loss: 1.931567 | Test Accuracy: 0.424015
Step 60 | Traini

In [ ]:
dict1 = {}
dict2 = []
for k, (v1, v2) in Train.predictions.items():
    dict1.update({k: v1})
    dict2.append(v2)
    

In [ ]:
Train.predictions = dict1
Train.results = dict2

In [ ]:
df_results = pd.DataFrame(Train.results)

In [ ]:
df_results.sort_values(by = 'test_score', ascending = False)

In [ ]:
pd.Panel(Train.predictions).to_pickle("dataset/tf_vae_only_nsl_kdd_predictions.pkl")
df_results.to_pickle("dataset/tf_vae_only_nsl_kdd_scores.pkl")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import itertools

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    np.set_printoptions(precision=4)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j].round(4),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

def plot(actual_value, pred_value):
    from sklearn.metrics import confusion_matrix
    cm_2labels = confusion_matrix(y_pred = pred_value, y_true = actual_value)
    plt.figure(figsize=[6,6])
    plot_confusion_matrix(cm_2labels, preprocess.output_columns_2labels, normalize = True,
                         title = Train.best_parameters)

In [ ]:
plot(actual_value = Train.actual_value, pred_value = Train.pred_value)